In [43]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")


In [3]:
prompt = ChatPromptTemplate.from_template("Describe in 5 lines about the topic {topic}")
output_parser = StrOutputParser()
chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
result = chain.invoke({"topic": "AI"})

c:\Maran\StudyMaterials\Git\LangChain\LCEL\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
result

{'topic': 'AI',
 'text': 'Artificial Intelligence (AI) refers to the simulation of human intelligence in machines designed to think and learn like humans. It encompasses various subfields, including machine learning, natural language processing, and robotics. AI applications range from virtual assistants and recommendation systems to autonomous vehicles and medical diagnostics. The technology has the potential to revolutionize industries, improve efficiency, and enhance decision-making. However, it also raises ethical concerns regarding privacy, job displacement, and bias in algorithms.'}

## Now we can use the LCEL instead of LLMChain

In [7]:
lcel_chain = prompt | llm | output_parser
result = lcel_chain.invoke({"topic":"Langchain Expression Language"})

In [8]:
result

'Langchain Expression Language (LEL) is a powerful tool designed for building and managing complex workflows in natural language processing applications. It allows users to define expressions that can manipulate and transform data seamlessly. LEL integrates with various data sources and APIs, enabling dynamic interactions and real-time data processing. Its syntax is user-friendly, making it accessible for both developers and non-developers. Overall, LEL enhances the flexibility and efficiency of language model applications by streamlining the expression of logic and data handling.'

## Use the Runnables in LCEL using the Vector DB in Chromadb

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

def get_vector_store(embedding_function=OpenAIEmbeddings(),
                     storage_directory="embeddings",
                     collection_name="langchain"):
    vectordb = Chroma(
        embedding_function=embedding_function,
        persist_directory=storage_directory,
        collection_name=collection_name
    )
    return vectordb

In [12]:
vectorstore_a = get_vector_store(collection_name='vectorstore_a')

In [13]:
vectorstore_a.add_texts(['Langchain is a framework for developing GENAI Apps',
                         'FastAPI helps to creates API application in python','Apple a day keeps the Doctor away',
                         'Dan born in Oct 7'])

['a151ba2e-1ba2-4639-8074-3208908e46ec',
 'f1e8de4e-df73-4ce3-9643-692e2d245cec',
 'f81b3bd5-a900-4943-b673-cc91f0c230ad',
 '41ba7f74-78df-4fc8-92de-04c826a6304b']

In [14]:
vectorstore_b = get_vector_store(collection_name='vectorstore_b')

In [15]:
vectorstore_b.add_texts(['Using LangChain we can easily connect to any Large Language Models',
                         'Apple contains good source of nutrients',
                         'Dan was born in 1983'])

['f8d612bf-3916-4f8a-94c3-2c5cc475b728',
 'a06b84ed-db96-4998-b39b-90995e83046c',
 '83bc1dbc-f9c6-4be4-8873-877ea54f009e']

In [20]:
len(vectorstore_a.get()["ids"]), len(vectorstore_b.get()["ids"])

(4, 3)

## Now use the Runnables

In [35]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

In [22]:
retriever_a = vectorstore_a.as_retriever()
retriever_b = vectorstore_b.as_retriever()

In [39]:
prompt_str="""
Answer the following question based on the provided context.
Context: {context}
Question: {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_str)

In [41]:
retrieval = RunnableParallel(
    {"context":retriever_a,"question":RunnablePassthrough()}

)
chain = retrieval| prompt | llm | output_parser

In [42]:
chain.invoke("Explain about Langchain")

'Langchain is a framework designed for developing Generative AI (GENAI) applications. It provides tools and components that facilitate the creation and deployment of applications that leverage generative AI technologies.'

In [30]:
updated_prompt = """
Please answer the question based on the following contexts. If the context does not contain the answer, please say "I don't know."
Context:
{context_a}
\n\n
{context_b}
\n\n
Question: {question}
\n
Answer:
"""
prompt = ChatPromptTemplate.from_template(updated_prompt)

In [31]:
retrieval_enhanced = RunnableParallel(
    {"context_a":retriever_a,
     "context_b":retriever_b,
     "question":RunnablePassthrough()
    }
)

chain = retrieval_enhanced | prompt | llm | output_parser

In [44]:
chain.invoke("Use of Langchain?")

'Langchain is a framework for developing GENAI Apps.'

## RunnableLambda
- This allows us convert and use our custom fuction in the Langchain Expression Language as a pipe operator.
- Below is the example

In [34]:
def formatted_output(data):
    print("Thanks for using our AI chat")
    print("Here is the output follows: ")
    print("\n\n")
    print(data)

In [36]:
add_formatted_output = RunnableLambda(formatted_output)

In [37]:
prompt = ChatPromptTemplate.from_template(
    "Tell me about the {Topic} in 2 lines"
)
chain = prompt | llm | output_parser | add_formatted_output

In [38]:
chain.invoke("Python FastApi")

Thanks for using our AI chat
Here is the output follows: 



FastAPI is a modern, high-performance web framework for building APIs with Python 3.6+ based on standard Python type hints. It enables automatic generation of interactive API documentation and is designed for speed and ease of use, making it ideal for building RESTful services.
